In [1]:
import numpy as np
import pandas as pd    

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
def relu(x):
    return np.maximum(0,x)

In [4]:
def deriv_relu(x):
    der=list()
    der.clear()
    for val in x:
        if val>0:
            der.append(1)
        else:
            der.append(0)
    return der

In [5]:
def initialize(x, neuron_num1, neuron_num2):
    np.random.seed(12345)
    w1=2*np.random.random((len(x.columns), neuron_num1))-1
    w2=2*np.random.random((neuron_num1, neuron_num2))-1
    w3=2*np.random.random((neuron_num2, 1))-1
    bias1=2*np.random.random()-1
    bias2=2*np.random.random()-1
    bias3=2*np.random.random()-1
    return w1, w2, w3, bias1, bias2, bias3

In [6]:
def forward_prop(w1, w2, w3, bias1, bias2, bias3, x):
    x=x.reshape(13,1)
    z1=np.dot(w1.T,x)+bias1
    out1=relu(z1)
    z2=np.dot(w2.T,out1)+bias2
    out2=relu(z2) 
    z3=np.dot(w3.T,out2)+bias3
    out3=relu(z3)
    return z1, out1, z2, out2, z3, out3 

In [7]:
def back_prop(w1, w2, w3, bias1, bias2, bias3, x, y, z1, out1, z2, out2, z3, out3, lr, j):
    x=x.reshape(13,1)
    
    err_3=2*(np.subtract(out3.sum(axis=0), y))
    dummy_3=np.array(deriv_relu(z3)).reshape(len(z3),1)
    dc_dw3=lr*np.dot(out2,dummy_3)*err_3
    w3=np.subtract(w3,dc_dw3)
    dc_db3=lr*err_3
    bias3-=dc_db3
    
    
    err_2=2*(np.subtract(out2.sum(axis=0), y)) # karenin turevi oldugu icin 2*(y_pred-y_act)
    dummy_2=np.array(deriv_relu(z2)).reshape(len(z2),1)
    dc_dw2=lr*np.dot(out1,dummy_2.T)*err_2
    w2=np.subtract(w2,dc_dw2)
    dc_db2=lr*err_2
    bias2-=dc_db2

    err_1= 2*(np.subtract(out1.sum(axis=0), y))
    dummy_1=np.array(deriv_relu(z1)).reshape(len(z1),1)
    dc_dw1=lr*np.dot(x,dummy_1.T)*err_1
    w1=np.subtract(w1,dc_dw1)
    dc_db1=lr*err_1
    bias1-=dc_db1
    return w1, w2, w3, bias1, bias2, bias3

In [8]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [9]:
columns=("CRIM","ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV")

In [10]:
ds=pd.read_csv("/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/housing.csv", sep='\s+', engine='python', names=columns)

In [11]:
target=ds.loc[:,'MEDV'].copy()
data=ds.drop('MEDV', axis=1).copy()

In [12]:
neuron_num1=7
neuron_num2=7

In [13]:
#data_normalized=(data.to_numpy()-min(data.to_numpy()))/(max(data.to_numpy())-min(data.to_numpy()))

In [14]:
w1, w2, w3, bias1, bias2, bias3 = initialize(data, neuron_num1, neuron_num2)

In [15]:
scaler=StandardScaler()
data_normalized=scaler.fit_transform(data)
target=target.to_numpy()

In [16]:
cutoff = int(len(data) * 0.8)
x_train, x_test = data_normalized[:cutoff], data_normalized[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [17]:
epochs=5
lr=0.001

In [18]:
output1=list()
output2=list()
output3=list()
z_1=list()
z_2=list()
z_3=list()
loss=0
loss_test=0

In [19]:
for e in range(epochs):
    output1.clear()
    output2.clear()
    output3.clear()
    z_1.clear()
    z_2.clear()
    z_3.clear()
    errors=0
    for i in range(len(y_train)):   
        z1, out1, z2, out2, z3, out3= forward_prop(w1, w2, w3, bias1, bias2, bias3, x_train[i])
        output1.append(out1)
        output2.append(out2)
        output3.append(out3)
        z_1.append(z1)
        z_2.append(z2)
        z_3.append(z3)
       # predicted[i]=output[i].sum(axis=0)
        w1, w2, w3, bias1, bias2, bias3 = back_prop(w1, w2, w3, bias1, bias2, bias3, x_train[i], y_train[i], z_1[i],
                                                    output1[i], z_2[i], output2[i], z_3[i], output3[i], lr, i+1)

In [20]:
print(w1.shape)

(13, 7)


In [21]:
print(w2.shape)

(7, 7)


In [22]:
print(w3.shape)

(7, 1)


In [23]:
for i in range(len(y_train)):
   # print(y_train[i], output[i].sum(axis=0))
    loss+=(y_train[i]-(output3[i].sum(axis=0)))
print(loss/(len(y_train)))

[1.02530266]


In [24]:
#for i in range(len(y_train)):
#    print(y_train[i], output3[i])

In [25]:
out_test=list()
for i in range(len(y_test)): 
    z_test1, out_test1, z_test2, out_test2, z_test3, out_test3=forward_prop(w1, w2, w3, bias1, bias2, bias3, x_test[i])
    out_test.append(out_test3)
    loss_test+=(y_test[i]-(out_test[i].sum(axis=0)))
print(loss_test/(len(y_test)))

[5.85534671]


In [26]:
for i in range(len(y_test)):
    print(y_test[i], out_test[i])

8.5 [[11.68170489]]
5.0 [[11.76763435]]
11.9 [[7.95872098]]
27.9 [[8.70454286]]
17.2 [[10.4284017]]
27.5 [[11.12704068]]
15.0 [[9.73792269]]
17.2 [[11.51088623]]
17.9 [[9.41962233]]
16.3 [[10.10379452]]
7.0 [[9.8032816]]
7.2 [[11.94870063]]
7.5 [[12.18801977]]
10.4 [[10.77757084]]
8.8 [[11.85729715]]
8.4 [[12.48904001]]
16.7 [[8.83253191]]
14.2 [[8.77359426]]
20.8 [[10.03885571]]
13.4 [[11.32450262]]
11.7 [[10.36443116]]
8.3 [[11.12836053]]
10.2 [[11.03005085]]
10.9 [[11.9956933]]
11.0 [[11.68479143]]
9.5 [[11.69522887]]
14.5 [[10.85665181]]
14.1 [[11.61772085]]
16.1 [[9.26137138]]
14.3 [[10.21361783]]
11.7 [[10.40697778]]
13.4 [[12.15696533]]
9.6 [[11.51159158]]
8.7 [[11.36953419]]
8.4 [[11.74496388]]
12.8 [[10.527468]]
10.5 [[11.68427368]]
17.1 [[9.96535987]]
18.4 [[8.71913562]]
15.4 [[9.65863399]]
10.8 [[11.29047668]]
11.8 [[11.79049184]]
14.9 [[9.4708923]]
12.6 [[9.25590534]]
14.1 [[9.62411899]]
13.0 [[10.10964574]]
13.4 [[11.00277876]]
15.2 [[9.76355766]]
16.1 [[9.68961494]]
17.8 